In [4]:
import tensorflow as tf
from keras.applications.resnet_v2 import ResNet50V2

In [5]:
import cv2
import numpy as np
import scipy.ndimage as ndi


In [6]:

# # 이미지 로드
# image_path = './open/train_imgs/BC_01_0001.png'  # 분할할 이미지 파일 경로
# image = cv2.imread(image_path)

# # 이미지를 LAB 색 공간으로 변환
# lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

# # A 채널 추출
# a_channel = lab_image[:, :, 1]

# # Otsu 임계값 적용
# _, thresholded_image = cv2.threshold(a_channel, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# # 분할 결과 출력
# cv2.imshow('Segmentation Result', thresholded_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [7]:
# 이미지 로드
image_path = './open/train_imgs/BC_01_0062.png'  # 분할할 이미지 파일 경로
image = cv2.imread(image_path)

In [8]:
def _get_masked_image(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    _, a, _ = cv2.split(lab)
    th = cv2.threshold(
        a, 127, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]

    mask = np.zeros_like(a)
    mask[a < th] = 1
    mask[a >= th] = 2
    mask = ndi.binary_fill_holes(mask-1)

    masked_image = np.zeros_like(image)
    masked_image[mask == 1] = image[np.where(mask == 1)]
    masked_image[mask == 0] = 255.

    return masked_image
def _crop_image(image):
    for w_pos in reversed(range(image.shape[1])):
        if (image[:, w_pos] == [255, 255, 255]).all():
            image = np.delete(image, w_pos, 1)
    for h_pos in reversed(range(image.shape[0])):
        if (image[h_pos, :] == [255, 255, 255]).all():
            image = np.delete(image, h_pos, 0)

    return image

In [9]:
def tile_image(image, tile_size, n_tiles):
    h, w, ch = image.shape

    pad_h, pad_w = (
        tile_size - h % tile_size) % tile_size, (tile_size - w % tile_size) % tile_size
    padding = [[pad_h//2, pad_h-pad_h//2],
               [pad_w//2, pad_w-pad_w//2], [0, 0]]
    image = np.pad(image, padding, mode='constant', constant_values=255)

    image = image.reshape(
        image.shape[0]//tile_size, tile_size, image.shape[1]//tile_size, tile_size, ch)
    tiles = image.transpose(
        0, 2, 1, 3, 4).reshape(-1, tile_size, tile_size, ch)

    if len(tiles) < n_tiles:
        padding = [[0, n_tiles-len(tiles)], [0, 0], [0, 0], [0, 0]]
        tiles = np.pad(tiles, padding, mode='constant', constant_values=255)

    idxs = np.argsort(tiles.reshape(tiles.shape[0], -1).sum(-1))[:n_tiles]
    tiles = tiles[idxs]

    return tiles


## 테스트

In [10]:
image1 = _get_masked_image(image)
# scale_percent = 20

# # 축소할 크기 계산
# width = int(image1.shape[1] * scale_percent / 100)
# height = int(image1.shape[0] * scale_percent / 100)

# # 이미지 축소
# resized_image1 = cv2.resize(image1, (width, height))

# 이미지 출력
cv2.imwrite('Resized Image1.png', image1)
cv2.waitKey(0)
cv2.destroyAllWindows()
# cv2.imshow('Segmentation Result', _get_masked_image(image))
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [11]:
image2 = _get_masked_image(image)
crop_2 = _crop_image(image2)

# scale_percent = 20

# # 축소할 크기 계산
# width = int(image2.shape[1] * scale_percent / 100)
# height = int(image2.shape[0] * scale_percent / 100)

# # 이미지 축소
# resized_image2 = cv2.resize(image2, (width, height))

# 이미지 출력
cv2.imwrite('Resized Image2.png', crop_2)
cv2.waitKey(0)
cv2.destroyAllWindows()
# cv2.imshow('Segmentation Result', _get_masked_image(image))
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [19]:
tiles = tile_image(crop_2, 224, 60)

In [20]:
import os
save_dir = './test_tiles'
file_prefix = 'test_tile'

# 타일 이미지 저장
for i, tile in enumerate(tiles):
    file_name = f'{file_prefix}{i}.png'
    file_path = os.path.join(save_dir, file_name)
    cv2.imwrite(file_path, tile)


In [ ]:
#https://github.com/lim-hyo-jeong/DACON-Breast-Cancer/blob/master/image_preprocessing.py
#참고해서 나누고 학습 진행 후 앙상블까지 끝내보기